In [ ]:
import numpy as np
import cv2 as cv
import math 
import os

rostro = cv.CascadeClassifier('haarcascade_frontalface_alt.xml')
cap = cv.VideoCapture("C:/Datasets/Sports/sportimages/F1-cortado.mp4")

output_dir = "C:/Datasets/Sports/sportimages/f1"
os.makedirs(output_dir, exist_ok=True)

i = 1000

# Funciones de transformación
def augment_image(img):
    """Devuelve una lista de imágenes transformadas."""
    imgs = []

    # Original
    imgs.append(img)

    # Volteo horizontal
    imgs.append(cv.flip(img, 1))

    # Volteo vertical
    imgs.append(cv.flip(img, 0))

    # Rotación 15°
    h, w = img.shape[:2]
    M = cv.getRotationMatrix2D((w/2, h/2), 15, 1)
    imgs.append(cv.warpAffine(img, M, (w, h)))

    # Rotación -15°
    M = cv.getRotationMatrix2D((w/2, h/2), -15, 1)
    imgs.append(cv.warpAffine(img, M, (w, h)))

    # Ajuste de brillo
    imgs.append(cv.convertScaleAbs(img, alpha=1.0, beta=30))  # más brillante
    imgs.append(cv.convertScaleAbs(img, alpha=1.0, beta=-30)) # más oscuro

    # Aumento de contraste
    imgs.append(cv.convertScaleAbs(img, alpha=1.5, beta=0))

    # Desenfoque gaussiano
    imgs.append(cv.GaussianBlur(img, (3, 3), 0))

    return imgs


while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    rostros = rostro.detectMultiScale(gray, 1.3, 5)

    frame = cv.resize(frame, (28, 21), interpolation=cv.INTER_AREA)

    if i % 5 == 0:
        augmented_frames = augment_image(frame)

        for j, img_aug in enumerate(augmented_frames):
            filename = os.path.join(output_dir, f"{i}_{j}.jpg")
            cv.imwrite(filename, img_aug)

    cv.imshow('Resultado', frame)
    i += 1

    k = cv.waitKey(1)
    if k == 27: 
        break

cap.release()
cv.destroyAllWindows()
